## Generate the observations

In [1]:
import numpy as np
import openturns as ot

In [2]:
ot.RandomGenerator.SetSeed(0)

In [3]:
def modelChaboche(X):
    strain,R,C,gamma = X
    #stress = R + C*(1-np.exp(-gamma*strain))
    stress = R - C*np.expm1(-gamma*strain)
    return [stress]

In [4]:
g = ot.PythonFunction(4, 1, modelChaboche) 

In [5]:
Strain = ot.Uniform(0,0.07)
unknownR = 750
unknownC = 2750
unknownGamma = 10
R = ot.Dirac(unknownR)
C = ot.Dirac(unknownC)
Gamma = ot.Dirac(unknownGamma)

Strain.setDescription(["Strain"])
R.setDescription(["R"])
C.setDescription(["C"])
Gamma.setDescription(["Gamma"])

Create the joint input distribution function.

In [6]:
inputRandomVector = ot.ComposedDistribution([Strain, R, C, Gamma])

Create the Monte-Carlo sample.

In [7]:
size = 5
inputSample = inputRandomVector.getSample(size)
outputStress = g(inputSample)
outputStress[0:5]

class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=5 dimension=1 description=[y0] data=[[1730.52],[2017.64],[998.458],[811.861],[1343.12]]

Generate observation noise.

In [8]:
stressObservationNoiseSigma = 40 # (Pa)
noiseSigma = ot.Normal(0.,stressObservationNoiseSigma)
sampleNoiseH = noiseSigma.getSample(size)
outputObservations = outputStress + sampleNoiseH

In [9]:
inputObservations = inputSample[:,0]

## Set the calibration parameters

Define the value of the reference values of the $\theta$ parameter. In the bayesian framework, this is called the mean of the *prior* gaussian distribution. In the data assimilation framework, this is called the *background*.

In [10]:
R = 700 # Exact : 750e6
C = 2500 # Exact : 2750e6
Gamma = 8. # Exact : 10
candidate = ot.Point([R,C,Gamma])

The following statement create the calibrated function from the model. The calibrated parameters Ks, Zv, Zm are at indices 1, 2, 3 in the inputs arguments of the model.

In [11]:
calibratedIndices = [1,2,3]
model = ot.ParametricFunction(g, calibratedIndices, candidate)

## Gaussian linear calibration

The standard deviation of the observations.

In [12]:
sigmaStress = 10. # (Pa)

Define the covariance matrix of the output Y of the model.

In [13]:
errorCovariance = ot.CovarianceMatrix(1)
errorCovariance[0,0] = sigmaStress**2

Defined the covariance matrix of the parameters $\theta$ to calibrate.

In [14]:
sigmaR = 0.1 * R
sigmaC = 0.1 * C
sigmaGamma = 0.1 * Gamma

In [15]:
parameterCovariance = ot.CovarianceMatrix(3)
parameterCovariance[0,0] = sigmaR**2
parameterCovariance[1,1] = sigmaC**2
parameterCovariance[2,2] = sigmaGamma**2
parameterCovariance

class=CovarianceMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[4900,0,0,0,62500,0,0,0,0.64]

## Calibration

The `GaussianLinearCalibration` class performs the gaussian linear calibration by linearizing the model in the neighbourhood of the prior.

In [16]:
algo = ot.GaussianLinearCalibration(model, inputObservations, outputObservations, candidate, parameterCovariance, errorCovariance)

The `run` method computes the solution of the problem.

In [17]:
algo.run()

In [18]:
calibrationResult = algo.getResult()

## Analysis of the results

The `getParameterMAP` method returns the maximum of the posterior distribution of $\theta$.

In [19]:
thetaStar = calibrationResult.getParameterMAP()
thetaStar

class=Point name=Unnamed dimension=3 values=[739.455,2746.96,10.6293]

We can compute a 95% confidence interval of the parameter $\theta^\star$. 

In [20]:
thetaPosterior = calibrationResult.getParameterPosterior()
thetaPosterior

class=Normal name=Normal dimension=3 mean=class=Point name=Unnamed dimension=3 values=[739.455,2746.96,10.6293] sigma=class=Point name=Unnamed dimension=3 values=[8.08963,143.471,0.593968] correlationMatrix=class=CorrelationMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[1,0.1796,-0.35705,0.1796,1,-0.974716,-0.35705,-0.974716,1]

In [21]:
covarianceThetaStar = thetaPosterior.getCovariance()
covarianceThetaStar

class=CovarianceMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[65.4421,208.449,-1.71562,208.449,20584,-83.0627,-1.71562,-83.0627,0.352798]

## Calibration based on Kalman matrix

In [22]:
parameterDimension = candidate.getDimension()

In [23]:
model.setParameter(candidate)

In [24]:
modelObservations = model(inputObservations)
modelObservations[0:5]

class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=5 dimension=1 description=[y0] data=[[1443.08],[1675.11],[882.391],[745.092],[1141.58]]

In [25]:
transposedGradientObservations = ot.Matrix(parameterDimension,size)
for i in range(size):
    g = model.parameterGradient(inputObservations[i])
    transposedGradientObservations[:,i] = g

In [26]:
gradientObservations = transposedGradientObservations.transpose()
gradientObservations[0:5,:]

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=5 columns=3 values=[1,1,1,1,1,0.297234,0.390046,0.0729565,0.0180369,0.176632,77.4648,94.2324,21.9462,5.58539,50.0072]

In [27]:
deltay = outputObservations - modelObservations
deltay = deltay.asPoint()
deltay[0:5]

class=Point name=Unnamed dimension=5 values=[364.348,343.228,89.6333,55.801,236.21]

Compute inverses of B and R.

In [28]:
B = ot.CovarianceMatrix(parameterCovariance)
IB = ot.IdentityMatrix(parameterDimension)
invB = B.solveLinearSystem(IB)

In [29]:
R = ot.CovarianceMatrix(size)
IR = ot.IdentityMatrix(size)
invR = R.solveLinearSystem(IR)

Calcule $A^{-1} = B^{-1} + J^T R^{-1} J = B^{-1} + J^T (J^T R^{-1})^T$.

Soit $C =J^T R^{-1}$. 

Cela implique $A^{-1} = B^{-1} + J^T C^T$.

In [30]:
C = gradientObservations.transpose() * invR

In [31]:
invA = invB + C * gradientObservations
invA

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[5.0002,0.954905,249.236,0.954905,0.277346,70.3148,249.236,70.3148,17895.7]

Calcule $K = A J^T R^{-1} = A C = (A^{-1})^{-1} C$.

In [32]:
K = invA.solveLinearSystem(C)

Calcule $\hat{\theta} = \mu  + K (y - H(\mu ))$.

In [33]:
thetaStar = candidate + K * deltay
thetaStar

class=Point name=Unnamed dimension=3 values=[732.338,2499.5,11.7235]

## Calcule la matrice de covariance

In [34]:
invA

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[5.0002,0.954905,249.236,0.954905,0.277346,70.3148,249.236,70.3148,17895.7]

Calcule la matrice $A$.

In [35]:
IA = ot.IdentityMatrix(parameterDimension)
A = invA.solveLinearSystem(IA)
A

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[0.653667,-0.0104657,-0.00906262,-0.0104611,0.000167491,0.000145267,-0.00906263,0.000145332,0.000181524]

## Autre calcul de la matrice de covariance

Un autre moyen de calculer la matrice de covariance.

In [36]:
L = IB - K * gradientObservations

In [37]:
covarianceThetaStar = K * R * K.transpose() + L * B * L.transpose()
covarianceThetaStar

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[16.552,996.545,-4.1461,996.545,62467.3,-259.323,-4.1461,-259.323,1.07672]

Calcul de A avec numpy.

In [38]:
Anp = ot.Matrix(np.linalg.inv(invA))
Anp

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[1.02814,23.4628,-0.106508,23.4628,1471.38,-6.10803,-0.106508,-6.10803,0.0255386]

## Par résolution d'un problème de moindres carrés linéaires

In [39]:
LB = B.computeCholesky()
LB

class=TriangularMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[70,0,0,0,250,0,0,0,0.8]

In [40]:
LR = R.computeCholesky()
LR[0:5,:]

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=5 columns=5 values=[1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1]

In [41]:
ILR = ot.IdentityMatrix(size)
invLR = LR.solveLinearSystem(ILR)
invLR[0:5,:]

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=5 columns=5 values=[1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1]

In [42]:
A = invLR * gradientObservations * LB
A[0:5,:]

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=5 columns=3 values=[70,70,70,70,70,74.3084,97.5114,18.2391,4.50922,44.158,61.9718,75.3859,17.557,4.46831,40.0058]

In [43]:
ITheta = ot.IdentityMatrix(parameterDimension)

In [44]:
llsqA = ot.Matrix(size+parameterDimension,parameterDimension)
llsqA[0:parameterDimension,0:parameterDimension] = ITheta
llsqA[parameterDimension:size+parameterDimension,0:parameterDimension] = llsqA[parameterDimension:size+parameterDimension,0:parameterDimension]-A

In [45]:
y = ot.Point(size+parameterDimension)
y[parameterDimension:size+parameterDimension] = y[parameterDimension:size+parameterDimension]-deltay

In [46]:
method = ot.SVDMethod(llsqA)
method

class=SVDMethod

In [47]:
deltaTheta = method.solve(y)
deltaTheta

class=Point name=Unnamed dimension=3 values=[0.0876305,-6.57209,13.1777]

In [48]:
thetaStar = ot.Matrix(LB) * deltaTheta + candidate
thetaStar

class=Point name=Unnamed dimension=3 values=[706.134,856.977,18.5421]

## Par résolution d'un problème de moindres carrés linéaires - v2

In [49]:
LB = B.computeCholesky()
LB

class=TriangularMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[70,0,0,0,250,0,0,0,0.8]

In [50]:
LR = R.computeCholesky()
LR[0:5,:]

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=5 columns=5 values=[1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1]

In [51]:
ILB = ot.IdentityMatrix(parameterDimension)
invLB = LB.solveLinearSystem(ILB)
invLB[0:5,:]

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[0.0142857,0,0,0,0.004,0,0,0,1.25]

In [52]:
invLRJ = LR.solveLinearSystem(gradientObservations)

In [53]:
llsqA = ot.Matrix(size+parameterDimension,parameterDimension)
llsqA[0:parameterDimension,0:parameterDimension] = invLB
llsqA[parameterDimension:size+parameterDimension,0:parameterDimension] = llsqA[parameterDimension:size+parameterDimension,0:parameterDimension]-invLRJ

In [54]:
invLRz = LR.solveLinearSystem(deltay)

In [55]:
y = ot.Point(size+parameterDimension)
y[parameterDimension:size+parameterDimension] = y[parameterDimension:size+parameterDimension]-invLRz

In [56]:
method = ot.SVDMethod(llsqA)
method

class=SVDMethod

In [57]:
deltaTheta = method.solve(y)
deltaTheta

class=Point name=Unnamed dimension=3 values=[6.13414,-1643.02,10.5421]

In [58]:
thetaStar = deltaTheta + candidate
thetaStar

class=Point name=Unnamed dimension=3 values=[706.134,856.977,18.5421]

In [59]:
method.getGramInverse()

class=CovarianceMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[1.02814,23.4628,-0.106508,23.4628,1471.38,-6.10803,-0.106508,-6.10803,0.0255386]